In [3]:
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix, issparse 

class ItemKNNRecommender:
    """
    Item-based kNN CF with:
      - user mean-centering
      - cosine similarity between items
      - optional shrinkage on similarities
      - top-k neighbors per item kept as a sparse matrix

    Parameters
    ----------
    k : int
        Number of nearest-neighbor items to keep per item.
    shrinkage : float
        Similarity shrinkage (>=0). Larger values damp similarities
        when co-rating counts are small. sim' = (n_ij/(n_ij+shrink))*sim
    """
    def __init__(self, k=50, shrinkage=100.0):
        self.k = int(k)
        self.shrinkage = float(shrinkage)
        self.user_means_ = None           # (n_users,)
        self.S_ = None                    # item-item similarity (csc_matrix)
        self.train_ = None                # original ratings CSR
        self.n_users_ = 0
        self.n_items_ = 0

    @staticmethod
    def _row_means_csr(X: csr_matrix):
        """Mean over nonzeros for each row; 0 if a user has no ratings."""
        counts = np.diff(X.indptr)
        sums = np.array(X.sum(axis=1)).ravel()
        means = np.zeros(X.shape[0], dtype=np.float32)
        nonzero_mask = counts > 0
        means[nonzero_mask] = (sums[nonzero_mask] / counts[nonzero_mask]).astype(np.float32)
        return means

    @staticmethod
    def _center_rows_inplace(X: csr_matrix, row_means):
        """Subtract row_means[u] from each nonzero entry in row u (inplace on a copy)."""
        X = X.copy()  # work on a copy
        # For each row, subtract its mean from its slice
        for u in range(X.shape[0]):
            start, end = X.indptr[u], X.indptr[u+1]
            if start < end:
                X.data[start:end] -= row_means[u]
        return X

    @staticmethod
    def _topk_by_col(S_csc: csc_matrix, k: int):
        """Keep top-k absolute values per column of a CSC sparse matrix."""
        S_csc = S_csc.copy()
        n_items = S_csc.shape[0]
        for j in range(n_items):
            start, end = S_csc.indptr[j], S_csc.indptr[j+1]
            col_indices = S_csc.indices[start:end]
            col_data = S_csc.data[start:end]
            if col_data.size > k:
                # Indices of top-k by absolute value
                top_idx = np.argpartition(np.abs(col_data), -k)[-k:]
                mask = np.zeros(col_data.size, dtype=bool)
                mask[top_idx] = True
                # Zero out everything else
                col_data[~mask] = 0.0
                S_csc.data[start:end] = col_data
        S_csc.eliminate_zeros()
        return S_csc

    def fit(self, X):
        """
        Fit on a user-item rating matrix.

        Parameters
        ----------
        X : array-like or sparse CSR of shape (n_users, n_items)
            Ratings matrix. Zeros/absent entries = no rating.
        """
        if not issparse(X):
            X = csr_matrix(np.asarray(X), dtype=np.float32)
        else:
            X = X.tocsr().astype(np.float32)

        self.train_ = X
        self.n_users_, self.n_items_ = X.shape

        # 1) User mean-centering
        self.user_means_ = self._row_means_csr(X)
        Xc = self._center_rows_inplace(X, self.user_means_)

        # 2) Cosine similarity between item columns: S = (Xc^T Xc) / (||i||*||j||)
        # Numerators:
        XtX = (Xc.T @ Xc).astype(np.float32)           # square (n_items x n_items) sparse
        # Denominator norms:
        diag = np.sqrt(XtX.diagonal().clip(min=1e-12)) # item L2 norms
        inv_diag = 1.0 / diag
        D_inv = csc_matrix((inv_diag, (np.arange(self.n_items_), np.arange(self.n_items_))),
                           shape=(self.n_items_, self.n_items_))
        S = D_inv @ XtX @ D_inv                        # normalized cosine similarities
        S.setdiag(0.0)
        S.eliminate_zeros()

        # 3) Apply shrinkage using co-rating counts:
        Xbin = X.copy()
        Xbin.data[:] = 1.0
        C = (Xbin.T @ Xbin).astype(np.float32)         # co-rating counts
        C.setdiag(0)
        if self.shrinkage > 0:
            # Align structures for elementwise update: use COO
            S = S.tocoo()
            # For every nonzero in S, scale by n_ij / (n_ij + shrink)
            # Fetch corresponding counts from C
            C_coo = C.tocsr()
            n_ij = C_coo[S.row, S.col].A.ravel()
            factor = n_ij / (n_ij + self.shrinkage)
            S.data = S.data * factor.astype(np.float32)
            S = S.tocsc()
        else:
            S = S.tocsc()

        # 4) Keep only top-k neighbors per item for efficiency
        self.S_ = self._topk_by_col(S, self.k)
        return self

    def _user_vector(self, u):
        """Return user's centered ratings as (indices, values)."""
        start, end = self.train_.indptr[u], self.train_.indptr[u+1]
        items = self.train_.indices[start:end]
        vals = self.train_.data[start:end] - self.user_means_[u]
        return items, vals

    def predict_one(self, u, i):
        """
        Predict rating for user u on item i.
        """
        if self.S_ is None:
            raise RuntimeError("Call fit() first.")
        if i < 0 or i >= self.n_items_:
            return float(self.user_means_[u])

        # neighbors of item i:
        start, end = self.S_.indptr[i], self.S_.indptr[i+1]
        neigh_idx = self.S_.indices[start:end]
        neigh_sim = self.S_.data[start:end]

        # items rated by user u:
        u_items, u_vals = self._user_vector(u)

        # intersect neighbors with user's rated items
        # build map for user's items -> centered value
        if u_items.size == 0 or neigh_idx.size == 0:
            return float(self.user_means_[u])

        # intersect via hashing
        u_map = dict(zip(u_items.tolist(), u_vals.tolist()))
        common_mask = np.array([j in u_map for j in neigh_idx], dtype=bool)
        if not common_mask.any():
            return float(self.user_means_[u])

        sims = neigh_sim[common_mask]
        vals = np.array([u_map[j] for j in neigh_idx[common_mask]], dtype=np.float32)

        denom = np.abs(sims).sum()
        if denom <= 1e-12:
            return float(self.user_means_[u])

        est = self.user_means_[u] + float((sims @ vals) / denom)
        return est

    def predict_user(self, u):
        """
        Vectorized predictions for all items for a single user (fast recommend()).
        Returns an array of shape (n_items,).
        """
        if self.S_ is None:
            raise RuntimeError("Call fit() first.")

        mean_u = self.user_means_[u]
        # Build a sparse vector of centered ratings for user u: (1 x n_items)
        start, end = self.train_.indptr[u], self.train_.indptr[u+1]
        cols = self.train_.indices[start:end]
        data = self.train_.data[start:end] - mean_u
        if cols.size == 0:
            return np.full(self.n_items_, mean_u, dtype=np.float32)

        r_u_centered = csc_matrix((data, (np.zeros_like(cols), cols)),
                                  shape=(1, self.n_items_), dtype=np.float32)

        # scores = r_u_centered @ S  (since S is item-item; here we want per-target item)
        # But our S_ is csc (cols = target items). We want scores for all target items:
        scores = (r_u_centered @ self.S_).toarray().ravel()

        # denom per target item = sum |sim(i, j)| over j in items rated by user
        # Equivalent to multiply with absolute S restricted to rated items
        absS = self.S_.copy()
        absS.data = np.abs(absS.data)
        denom = (np.abs(r_u_centered) @ absS).toarray().ravel()
        
        with np.errstate(divide='ignore', invalid='ignore'):
            delta = np.zeros_like(scores)
            mask = denom > 1e-12
            delta[mask] = scores[mask] / denom[mask]
        return mean_u + delta

    def recommend(self, u, N=10, filter_seen=True):
        """
        Recommend top-N items for user u.

        Returns
        -------
        list of (item_id, score)
        """
        preds = self.predict_user(u)
        if filter_seen:
            start, end = self.train_.indptr[u], self.train_.indptr[u+1]
            seen = set(self.train_.indices[start:end].tolist())
        else:
            seen = set()

        candidates = [(i, preds[i]) for i in range(self.n_items_) if i not in seen]
        candidates.sort(key=lambda x: x[1], reverse=True)
        return candidates[:N]

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from scipy.sparse import coo_matrix, csc_matrix, csr_matrix, issparse

def fit_fast_knn(self, X):
    """
    Faster alternative fit: build item-item top-k graph using sklearn's
    NearestNeighbors (cosine) directly, avoiding full XtX.
    Includes robust NaN/Inf cleaning.
    """
    import numpy as np

    # Ensure CSR float32 and clean any non-finite values up front
    if not issparse(X):
        X = csr_matrix(np.asarray(X), dtype=np.float32)
    else:
        X = X.tocsr().astype(np.float32)
    if X.data.size:
        X.data[~np.isfinite(X.data)] = 0.0

    self.train_ = X
    self.n_users_, self.n_items_ = X.shape

    # --- user mean-centering ---
    self.user_means_ = self._row_means_csr(X)
    Xc = self._center_rows_inplace(X, self.user_means_)

    # Any NaN/Inf after centering? (shouldn't happen, but guard anyway)
    if Xc.data.size:
        Xc.data[~np.isfinite(Xc.data)] = 0.0

    # --- item matrix for cosine kNN ---
    # columns = items; normalize columns (L2) so dot == cosine
    Xi = Xc.tocsc(copy=True)
    if Xi.data.size:
        Xi.data[~np.isfinite(Xi.data)] = 0.0
    Xi = normalize(Xi, axis=0, norm="l2", copy=False)  # column-wise L2
    Xi = Xi.T.tocsr()  # shape: (n_items, n_users)

    # --- exact kNN over items (cosine distance) ---
    k_eff = min(self.k + 1, self.n_items_)  # +1 for self-neighbor
    nn = NearestNeighbors(
        n_neighbors=k_eff,
        algorithm="brute",
        metric="cosine",
        n_jobs=-1
    )
    nn.fit(Xi)
    distances, indices = nn.kneighbors(Xi, return_distance=True)  # (n_items, k_eff)

    sims = 1.0 - distances.astype(np.float32)  # cosine sim
    sims = sims[:, 1:]                         # drop self
    indices = indices[:, 1:]

    # --- optional shrinkage ---
    if self.shrinkage > 0:
        Xbin = self.train_.copy().astype(np.float32)
        if Xbin.data.size:
            Xbin.data[:] = 1.0
        Xbin = Xbin.tocsc()

        rows, cols, data = [], [], []
        for j in range(self.n_items_):
            neigh = indices[j]
            if neigh.size == 0:
                continue
            # co-ratings count n_ij for (j, neigh)
            #cj = Xbin[:, j].T @ Xbin[:, neigh]            # 1 x k dense
            #n_ij = np.asarray(cj).ravel().astype(np.float32)
            # co-ratings count n_ij for (j, neigh)
            # (1 x users) @ (users x k) -> (1 x k) dense
            cj = (Xbin[:, j].T @ Xbin[:, neigh]).toarray().ravel()
            n_ij = cj.astype(np.float32)

            shrink_factor = n_ij / (n_ij + self.shrinkage)
            s = sims[j] * shrink_factor
            rows.extend(neigh.tolist())
            cols.extend([j] * len(neigh))
            data.extend(s.tolist())
        S = coo_matrix((data, (rows, cols)), shape=(self.n_items_, self.n_items_)).tocsc()
    else:
        rows = indices.ravel()
        cols = np.repeat(np.arange(self.n_items_), indices.shape[1])
        data = sims.ravel()
        S = coo_matrix((data, (rows, cols)), shape=(self.n_items_, self.n_items_)).tocsc()

    S.setdiag(0.0)
    S.eliminate_zeros()

    # safety: keep top-k per column
    self.S_ = self._topk_by_col(S, self.k)
    return self

# ---------------------- Example usage ----------------------
if __name__ == "__main__":
    import pandas as pd
    import numpy as np

    df = pd.read_excel("train-1.xlsx")
    # Keep only item columns; replace NaN/±Inf with 0
    items_df = df.iloc[:, 1:].replace([np.inf, -np.inf], np.nan).fillna(0.0)

    R = items_df.to_numpy(dtype=np.float32)

    print("R shape:", R.shape, "| NaNs:", np.isnan(R).sum(), "| Infs:", np.isinf(R).sum())
    #print("Shape of R:", R.shape)
    #print("First 5 rows:\n", R[:5])
    # Monkey-patch or add as a method in your class:
    ItemKNNRecommender.fit = fit_fast_knn

    model = ItemKNNRecommender(k=50, shrinkage=100).fit(R)
    
    # --- Recommend top-20 items for all users ---
    all_recommendations = {}

    for u in range(R.shape[0]):
        recs = model.recommend(u, N=20)
        all_recommendations[u] = recs  # store (item_id, score) list per user

    # --- Optionally, save to a CSV file ---
    recommendation_list = []
    for u, recs in all_recommendations.items():
        for item_id, score in recs:
            recommendation_list.append([u, item_id, score])

    df_recs = pd.DataFrame(recommendation_list, columns=["user_index", "item_index", "predicted_score"])
    df_recs.to_csv("recommendations_top20.csv", index=False)
    print("\nSaved all recommendations to 'recommendations_top20.csv'")

R shape: (31667, 1848) | NaNs: 0 | Infs: 0

Saved all recommendations to 'recommendations_top20.csv'


In [ ]:
import pandas as pd

# Load your original file (user_index, item_index, predicted_score)
df = pd.read_csv("recommendations_top20.csv")

# Sort by user and score (just to ensure correct ranking)
df = df.sort_values(by=["user_index", "predicted_score"], ascending=[True, False])

# Group by user and collect top-20 item indices into lists
top_items = (
    df.groupby("user_index")["item_index"]
    .apply(lambda x: x.head(20).tolist())
    .reset_index()
)

# Expand each list into separate columns
top_items_expanded = top_items["item_index"].apply(pd.Series)
top_items_expanded.columns = [f"item_{i+1}" for i in range(top_items_expanded.shape[1])]

# Combine user_index and the 20 recommended items
final_df = pd.concat([top_items["user_index"], top_items_expanded], axis=1)

# Save to new CSV
final_df.to_csv("recommendations_top20_wide.csv", index=False)

print("✅ Saved to 'recommendations_top20_wide.csv'")
print(final_df.head())